In [ ]:
from z3 import *
import time
import math

DA FARE:
1. Create benchmark suite
2. Documentare il codice

## Utils

In [ ]:
def millisecs_left(t, timeout):
    return int((timeout - t) * 1000)

In [ ]:
#TODO: sposta gli utils tutti qua su
def flatten(matrix):
    return [e for row in matrix for e in row]

#### Conversion from int to bin

In [ ]:
def num_bits(x):
    return math.floor(math.log2(x)) + 1

In [ ]:
def int_to_bin(x, digits):
    x_bin = [(x%(2**(i+1)) // 2**i)==1 for i in range(digits-1,-1,-1)]
    return x_bin

def bin_to_int(x):
    n = len(x)
    x = sum(2**(n-i-1) for i in range(n) if x[i] == 1)
    return x

## Encodings

#### At most/least one & exactly one

In [ ]:
def at_least_one_seq(bool_vars):
    return Or(bool_vars)

def at_most_one_seq(x, name):
    # bool_vars renamed to x for simplicity
    n = len(x)
    if n == 1:
        return True
    s = [Bool(f"s_{i}_{name}") for i in range(n-1)]     # N.B. len(s) = n-1 != n = len (x)
    
    clauses = []
    clauses.append(Or(Not(x[0]), s[0]))                 # x[0] -> s[0] (s[i] modeled as: s[i] is true if the SUM UP TO index i IS 1!) i.e. x_1 -> s_1 in math notation
    for i in range(1, n-1):
        clauses.append(Or(Not(x[i]), s[i]))             # these two clauses model (x[i] v s[i-1]) -> s[i]
        clauses.append(Or(Not(s[i-1]), s[i]))
        clauses.append(Or(Not(s[i-1]), Not(x[i])))      # this one models s[i-1] -> not x[i]
    clauses.append(Or(Not(s[-1]), Not(x[-1])))          # s[n-2] -> not x[n-1]  i.e. s_(n-1) -> s_n in the mathematical notation (1-based)
    return And(clauses)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

#### Operations on binary numbers

In [ ]:
def equal(v, u):
    assert(len(v) == len(u))
    return And([v[k]==u[k] for k in range(len(v))])

def all_false(v):
    return And([Not(v[k]) for k in range(len(v))])

def leq_same_digits(v, u, digits):
    """Encoding of v <= u, implementation with digits fixed

    Args:
        v (list[bool]): binary representation of v
        u (list[bool]): binary representation of u
        digits (int): number of digits of v and u

    Returns:
        (Z3-expression): encoding of v <= u in binary considering their {digits} most significant bits
    """
    assert(len(v) == len(u) and len(u) == digits)
    if digits == 1:
        return Or(v[0]==u[0], And(Not(v[0]), u[0]))
    else:
        return Or(And(Not(v[0]), u[0]),
                  And(v[0]==u[0], leq_same_digits(v[1:], u[1:], digits-1)))


def leq(v, u):
    """Encoding of v <= u, implementation with different digits btw v and u

    Args:
        v (list[bool]): binary representation of v
        u (list[bool]): binary representation of u

    Returns:
        (Z3-expression): encoding of v <= u in binary
    """
    digits_v = len(v)
    digits_u = len(u)

    if digits_v == digits_u:
        return leq_same_digits(v, u, digits_v)
    elif digits_v < digits_u:
        delta_digits = digits_u - digits_v
        return Or(Or(u[:delta_digits]),
                  leq_same_digits(v, u[delta_digits:], digits_v))
    else:
        delta_digits = digits_v - digits_u
        return And(all_false(v[:delta_digits]), leq_same_digits(v[delta_digits:], u, digits_u))


    # if delta_digits == 0:   # i.e. digits_v == digits_u
    #     return leq_same_digits(v, u, digits_v)
    # else:
    #     return Or(Or(u[:delta_digits]), leq_same_digits(v, u[delta_digits:], digits_v))


In [ ]:
def sum_bin_same_digits(a_bin, b_bin, d_bin, digits, name):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, each number having {digits} num of bits

    Args:
        a_bin (list[bool]): binary representation of a
        b_bin (list[bool]): binary representation of b
        d_bin (list[bool]): binary representation of d
        digits (int): number of bits of each number 
        name (str): string to identify carry boolean variables

    Returns:
        formula (Z3-expression): formula representing SAT encoding of binary sum
        c[0] (Bool): last carry of binary encoding
    """
    # c_k represents carry at bit position k
    c = [Bool(f"c_{k}_{name}") for k in range(digits+1)]
    c[-1] = False

    clauses = []
    for k in range(digits-1,-1,-1):
        # clauses.append((a_bin[k] == b_bin[k]) == (c[k+1] == d_bin[k]))    #TODO: check if it is faster
        clauses.append(d_bin[k] == Or(And(a_bin[k], b_bin[k], c[k+1]), And(a_bin[k], Not(b_bin[k]), Not(c[k+1])),
                                And(Not(a_bin[k]), b_bin[k], Not(c[k+1])), And(Not(a_bin[k]), Not(b_bin[k]), c[k+1])))
        clauses.append(c[k] == Or(And(a_bin[k],b_bin[k]), And(a_bin[k],c[k+1]), And(b_bin[k],c[k+1])))

    formula = And(clauses)
    return (formula, c[0])

def sum_bin(a_bin, b_bin, d_bin, name):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, with digits(a_bin) <= digits(b_bin) == digits(d_bin)

    Args:
        a_bin (list[bool]): binary representation of a
        b_bin (list[bool]): binary representation of b
        d_bin (list[bool]): binary representation of d
        name (str): string to identify carry boolean variables

    Returns:
        (Z3-expression): formula representing SAT encoding of binary sum
    """
    digits_a = len(a_bin)
    digits_b = len(b_bin)
    digits_d = len(d_bin)
    assert(digits_a <= digits_b and digits_b == digits_d)

    delta_digits = digits_b - digits_a

    if delta_digits == 0:
        formula, last_carry = sum_bin_same_digits(a_bin, b_bin, d_bin, digits_a, name)
        return And(formula, Not(last_carry))    # imposing no overflow

    sub_sum_formula, last_carry = sum_bin_same_digits(a_bin, b_bin[delta_digits:], d_bin[delta_digits:], digits_a, name)
    c = [Bool(f"c_propagated_{k}_{name}") for k in range(delta_digits)] + [last_carry]
    c[0] = False     # imposing no further overflow

    clauses = []
    for k in range(delta_digits-1, -1, -1):
        clauses.append(d_bin[k] == Xor(b_bin[k], c[k+1]))
        clauses.append(c[k] == And(b_bin[k], c[k+1]))

    return And(And(clauses), sub_sum_formula)

In [ ]:
def conditional_sum_K_bin(x, alpha, delta, name):
    """Encodes into a SAT formula the constraint {delta = sum_over_j(alpha[j] | x[j] == True)}

    Args: 
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        delta (list[Bool]): list of Z3 Variables, which will be constrained to represent the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    
    """
    n = len(x)
    digits = len(delta)

    # matrix containing temporary results of sum_bin
    d = [[Bool(f"d_{j}_{k}_{name}") for k in range(digits)] for j in range(n-1)]    # j = 1..n-1 because last row will be delta
    d.append(delta)

    clauses = []

    # row 0
    diff_digits = digits - len(alpha[0])
    assert(diff_digits >= 0)
    clauses.append( And(Implies(x[0], And(all_false(d[0][:diff_digits]), equal(d[0][diff_digits:], alpha[0]))), # If x[0] == 1 then d_0 == alpha_0 (with eventual padding of zeros)
                        Implies(Not(x[0]), all_false(d[0]))))              # elif x[0] == 0 then d_0 == [0..0]

    # row j>1
    for j in range(1,n):
        clauses.append( And(Implies(x[j], sum_bin(alpha[j], d[j-1], d[j], f"{name}_{j-1}_{j}")),           # If c_j == 1 then d_j == d_j-1 + alpha_j 
                            Implies(Not(x[j]), equal(d[j], d[j-1]))))
        
    return And(clauses)

#### Linear Integer constraint over binary numbers

In [ ]:
# TODO: it's not used anymore now, can remove it now! ...or better, don't remove it and implement PB on it to optimize it
# Then find a way to also make it return the result of the sum as a Z3-Variable so that can impose symmetry breaking constraint on it!
def LinearInteger_bin(x, alpha, beta, name):
    """Encodes into a SAT formula the linear constraint {sum_over_j(alpha[j] | x[j] == True) <= beta}

    Args: 
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        beta (list[bool]): binary representing the known term, i.e. the upper bound for the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    """
    n = len(x)
    digits = len(beta)

    clauses = []


    sum_result = [Bool(f"LIsum_result_{k}_{name}") for k in range(digits)]
    clauses.append(conditional_sum_K_bin(x, alpha, sum_result, name))
    
    clauses.append(leq(sum_result, beta)) 

    formula = And(clauses)
    return formula



#### Functions to display solution

In [ ]:
def displayMCP(orders, distances_bin, obj_value, assignments=None):
    distances = [bin_to_int(d) for d in distances_bin]

    print(f"-----------Objective value: {obj_value}-----------")
    print(f"------------------Routes-----------------")
    if assignments is None:
        m = len(orders)
        n = len(orders[0])
        for i in range(m):
            visited = [n]
            for t in range(n):
                found = False
                for j in range(n):
                    if orders[i][j][t]:
                        visited.append(j)
                        found = True
                if not found:
                    break
            visited.append(n)

            print('Origin --> ' + ' --> '.join([str(vertex) for vertex in visited[1:-1]]) + f' --> Origin: travelled {distances[i]}')

    else: 
        m = len(assignments)
        n = len(assignments[0])
        t = 0
        i = 0
        visited = [n]
        while t < n:
            for j in range(n):
                if orders[j][t]:
                    if assignments[i][j]:
                        visited.append(j)
                    else:
                        visited.append(n)
                        print('Origin --> ' + ' --> '.join([str(vertex) for vertex in visited[1:-1]]) + f' --> Origin: travelled {distances[i]}')            
                        visited = [n, j]
                        i += 1
                    break
            t += 1
        visited.append(n)
        print('Origin --> ' + ' --> '.join([str(vertex) for vertex in visited[1:-1]]) + f' --> Origin: travelled {distances[i]}')            



#### Functions to display matrices of tensor and dist

In [ ]:
def evaluate(model, bools):
    if not isinstance(bools[0], list):
        return [1 if model.evaluate(b) else 0 for b in bools]
    return [evaluate(model, b) for b in bools]

#### Function to check that routes are all Hamiltonian cycles

In [ ]:
def check_all_hamiltonian(tensor):
    m = len(tensor)
    for i in range(m):
        if not check_hamiltonian(tensor[i]):
            return False
    return True

def check_hamiltonian(matrix):
    n = len(matrix)
    visited = [0] * n
    v = n-1
    while visited[v] == 0:
        visited[v] = 1
        for k in range(n):
            if matrix[v][k] == True:
                v = k
                break
    num_vertices = sum(row.count(True) for row in matrix)
    return (sum(visited) == num_vertices)


#### Objective function section

In [ ]:
def distances_def_constraint(distances, flat_r, flat_D_bin):
    """Defines the set of binary numbers {distances} as the sum of the binary numbers in {flat_D_bin} whose respective element in {flat_r} is True
        i.e. distances[i] = sum_over_j(flat_D_bin[j] | (flat_r[i])[j] == True)  forall i

    Args:
        distances (list[list[Bool]]): distances[i] will be constrained to be the sum of flat_D_bin[j] with respective flat_r[i][j] == True
        flat_r (list[list[Bool]]]): list of flattened matrices of Z3 Bool variables, each matrix flat_r[i] originally (i.e. before flattening) describing the route of courier i
        flat_D_bin (list[list[bool]]): flattened matrix of distances D, with each element converted to a binary as a list[bool]
    
    Returns:
        (Z3-expression): encoding of the definition of distances
    """
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    return And(clauses)    


def obj_function(model, distances):
    m = len(distances)
    maxdist = -1
    for i in range(m):
        dist = bin_to_int([1 if model.evaluate(distances[i][j]) else 0 for j in range(len(distances[i]))])
        maxdist = max(maxdist, dist)
    return maxdist

def AllLessEq_bin(distances, obj_value_bin):
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(leq(distances[i], obj_value_bin))

    return And(clauses)

def AtLeastOneGreaterEq_bin(distances, obj_value_bin):
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(leq(obj_value_bin, distances[i]))

    return Or(clauses)

## Implicit constraints

Assumpions that we can make:
1. $n >= m$
2. The distance matrix $D$ is quasimetric (more info [here](https://proofwiki.org/wiki/Definition:Quasimetric)) => Giving an item to courier who has none is always "same or better" than giving 2 to another courier and none to this one

The assumptions above give us the implicit constraint **"Every courier has at least one item assigned to it"**

## Symmetry breaking constraints

TODO: spiega symmetry breaking constraints

In [ ]:
def sort_decreasing(matrix):
    """Returns constraint that the binary numbers represented by the rows of {matrix} are sorted in decreasing order

    Args:
        matrix (list[list[bool]]): matrix[i] represents an integer in binary

    Returns:
        (Z3-expression): constraint representing the decreasing ordering of binary numbers in the rows of matrix
    """
    m = len(matrix)
    clauses = []
    for i in range(m-1):
        clauses.append(leq(matrix[i+1], matrix[i]))
    return And(clauses)


## Models

* Model 1: r and t matrices, optional symmetry breaking on loads
* Model 2: r, r_compressed and t_compressed matrices (s.t. to reduce encoding time), optional symmetry breaking on both loads and t_compressed

All implement both Linear and Binary Optimization search

In [ ]:
def multiple_couriers_planning_1(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)
                    
    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)]
    # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    solver = Solver()


    ## CONSTRAINTS
    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)
        # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
        solver.add(sort_decreasing(courier_loads))
        # Break symmetry within same load amounts, i.e.:
        # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
        # i.e. the second courier will be the one assigned to the route containing the item with lower index j
        for i in range(m - 1):
            solver.add(
                Implies(equal(courier_loads[i], courier_loads[i + 1]),
                        leq(a[i], a[i + 1])))

    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

    # TODO: remove
    print(f"Finished constraint 1 at time {round(time.time() - start_time, 1)}s")


    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
        solver.add(leq(courier_loads[i], l_bin[i]))

    # TODO: remove
    print(f"Finished constraint 2 at time {round(time.time() - start_time, 1)}s")


    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    for i in range(m):
        solver.add(at_least_one_seq(a[i]))

    # TODO: remove
    print(f"Finished constraint 3 at time {round(time.time() - start_time, 1)}s")


    # Constraint 4: routes

    for i in range(m):
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k)
        solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin

        # Constraint 4.4: HAMILTONIAN CYCLES
        # Uses matrix t[i] in order to check that the adjacency matrix r[i] represents a Hamiltonian cycle
        for j in range(n):
            # assignments implications
            solver.add(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) => [t_ijk = True, for some k] (i delivers j at some time k)
            solver.add(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) => [t_ijk = False, forall k] (i never delivers j)

            # can't deliver two items at the same time
            solver.add(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

            # 1st trip
            solver.add(r[i][n][j] == t[i][j][0]) # r_inj (i rides n (origin) --> j) => t_ij0 (i delivers j as 1st item)

            # successive trips
            for h in range(1, n):
                # The next two constraints make a double implication together: (Exists k s.t. i came from that k to j AND he delivered k at time h-1) <==> (j delivered at time h)
                # ==>
                for k in range(n):
                    solver.add(Implies(And(r[i][k][j], t[i][k][h-1]), t[i][j][h]))  # r_i,k,j AND t_i,k,h-1 (i goes from k to j AND he delivered k at time h-1) => t_i,j,h (j delivered at time h)

                # <==
                solver.add(Implies(t[i][j][h], Or([And(r[i][k][j], t[i][k][h-1]) for k in range(n)])))       # if t_i,j,h (j delivered at time h) => [r_i,k,j AND t_i,k,h-1. for some k] (i came from some k to j AND he delivered that k at time h-1)

    # TODO: remove
    print(f"Finished constraint 4 + Hamiltonian at time {round(time.time() - start_time, 1)}s")


    ## OPTIMIZATION SEARCH

    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]


    # Constraint 5: distances travelled by each courier
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])
    # TODO: maybe we can find a tighter upper bound on obj func, i.e. start from a naive solution, that is:
    # starting from the courier with lowest load, assign to him as many objects as possible (starting from the one with lowest load
    # going to the heaviest one, i.e. sort s first! BEWARE, NEED TO KEEP ALSO INITIAL INDEX IN ORDER TO USE PROPER INDEX IN D MATRIX) When he can't take any more object: continue with next courier etc. -> end up with a possible assignment
    # Then compute distances (can do also while assign objects to couriers! All in a single pass) and find the maximum.
    # Complexity: O(n) NICE!
    # Question: ask if you can do this! Pro: would make binary search start way lower! Also could add the initial constraint AllLessEq(distances, upper_bound) because right now doing so it's useless
    # TODO: Do as line above, add constriant AllLessEq(distances, upper_bound)
    # IMP: note that such a solution would NOT be possible if the ACTUAL LOADS ARE NOT SORTED! TODO: find an initial solution that satisfies this!
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])

    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    # definition of distances using constraints
    for i in range(m):
        solver.add(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    # TODO: remove
    print(f"Finished distances def constraint at time {round(time.time() - start_time, 1)}s")


    model = None
    obj_value = None
    encoding_time = time.time()
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")

    timeout = encoding_time + timeout_duration

    # TODO: remove
    # if now >= timeout:
    #     return "Encoding time too slow"

    solver.push()

    if search == 'Linear':

        solver.set('timeout', millisecs_left(time.time(), timeout))
        while solver.check() == z3.sat:

            model = solver.model()
            obj_value = obj_function(model, distances)
            print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")


            if obj_value <= lower_bound:
                break

            upper_bound = obj_value - 1
            upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            solver.pop()
            solver.push()

            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))


    elif search == 'Binary':

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T = evaluate(model, t)
        Dists = evaluate(model, distances)
        displayMCP(T, Dists, obj_value)

    return (obj_value, solving_time)


In [ ]:
def multiple_couriers_planning_2(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 2 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or now to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)
                    
    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    r_compressed = [[Bool(f"r_compressed_{j}_{k}") for k in range(n+1)] for j in range(n+1)]
    # r_compressed_jk = 1 indicates that some courier moves from delivery point j to delivery point k


    t_compressed = [[Bool(f"delivers_{j}_as_{h}-th") for h in range(n+1)] for j in range(n)]
    # t_compressed_jh = 1 iff object j is delivered as h-th element (i.e. at TIME=h, use of time as order) considering that
    # couriers leave one at a time and only once the previous one has returned to the origin point
    # last column of t_compressed is imposed to have all 0s in order to guarantee no cycles that exclude the origin
    

    solver = Solver()


    ## CONSTRAINTS
    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)
        # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
        solver.add(sort_decreasing(courier_loads))
        # Break symmetry within same load amounts, i.e.:
        # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a, 
        # i.e. the second courier will be the one assigned to the route containing the item with lower index j
        for i in range(m-1):
            solver.add(Implies(equal(courier_loads[i], courier_loads[i+1]), leq(a[i], a[i+1])))

    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

    # TODO: remove
    print(f"Finished constraint 1 at time {round(time.time() - start_time, 1)}s")


    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
        solver.add(leq(courier_loads[i], l_bin[i]))        

    # TODO: remove
    print(f"Finished constraint 2 at time {round(time.time() - start_time, 1)}s")


    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    for i in range(m):
        solver.add(at_least_one_seq(a[i]))

    # TODO: remove
    print(f"Finished constraint 3 at time {round(time.time() - start_time, 1)}s")




    # Constraint 4: routes
    for i in range(m):
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k)
        solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin
    
    # TODO: remove
    print(f"Finised constraint 4 at time {round(time.time() - start_time, 1)}s")


    # Constraint 4.4: HAMILTONIAN CYCLES
    # Derive matrix r_compressed from tensor r by "compressing" it over the couriers-dimension (using Or operator)
    # Derive matrix t_compressed from r_compressed

    # r_compressed def
    for j in range(n+1):
        for k in range(n+1):
            solver.add(Implies(Or([r[i][j][k] for i in range(m)]), r_compressed[j][k]))
            solver.add(Implies(all_false([r[i][j][k] for i in range(m)]), Not(r_compressed[j][k])))

    # t_compressed def
    for j in range(n):
        solver.add(exactly_one_seq(t_compressed[j], f'no_double_delivery_{j}'))
        solver.add(exactly_one_seq([t_compressed[h][j] for h in range(n)], f'no_contemporary_delivery_{j}'))
    solver.add(all_false([t_compressed[h][n] for h in range(n)]))

    # 1st trip
    solver.add(equal(r[0][n][:n], [t_compressed[j][0] for j in range(n)]))  # start from deliveries of 1st courier in t_compressed

    # successive trips
    for h in range(1, n+1):
        for j in range(n):
            # (Exists k s.t. i came from that k to j AND he delivered k at time h-1) ==> (j delivered at time h)
            for k in range(n):
                solver.add(Implies(And(r_compressed[k][j], t_compressed[k][h-1]), t_compressed[j][h]))  # r_i,k,j AND t_i,k,h-1 (i goes from k to j AND he delivered k at time h-1) => t_i,j,h (j delivered at time h)

            # Don't include the inverse implication (<==) because won't be satisfied for deliveries of two different couriers

    # TODO: remove
    print(f"Finised constraint Hamiltonian (r_compressed and t_compressed def) at time {round(time.time() - start_time, 1)}s")


    # Constraint 4.4.1: symmetry breaking constraint over the definition of t_compressed
    if symmetry_breaking:
        # couriers_orders_ki = 1 iff object delivered at time k is delivered by courier i
        couriers_orders = [[Bool(f"courier_{i}_delivers_{k}-th_object") for i in range(m)] for k in range(n)]
        
        for k in range(n):
            solver.add(exactly_one_seq(couriers_orders[k], f'only_one_courier_delivers_at_time_{k}'))

        for i in range(m):
            for j in range(n):
                for k in range(n):
                    # (object j assigned to i AND object j is delivered at time k)
                    # ==> object delivered at time k is delivered by courier i
                    solver.add(Implies(And(a[i][j], t_compressed[j][k]), couriers_orders[k][i]))

        # put in t_compressed first all objects delivered by courier 0, then 1, then 2, ...
        for k in range(n-1):
            for i in range(m-1):
                # next object is either still of courier i or of courier i+1
                solver.add(Implies(couriers_orders[k][i], Xor(couriers_orders[k+1][i], couriers_orders[k+1][i+1])))

                # direct search: if there is a change in couriers, start (in t_compressed) from 1st object delivered by next courier
                solver.add(Implies(And(couriers_orders[k][i], couriers_orders[k+1][i+1]), equal([t_compressed[j][k+1] for j in range(n)], r[i+1][n][:n])))
            # if reached last courier, all next elements are delivered by him
            solver.add(Implies(couriers_orders[k][m-1], couriers_orders[k+1][m-1]))

    # TODO: remove
    print(f"Finished constraint 'Symmetry breaking on t_compressed' at time {round(time.time() - start_time, 1)}s")



    ## OPTIMIZATION SEARCH

    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]


    # Constraint 5: represent as Bool binary number the distances travelled by each courier
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])     # TODO: maybe we can find a tighter upper bound on obj func
    # TODO: maybe we can find a tighter upper bound on obj func, i.e. start from a naive solution, that is:
    # starting from the courier with lowest load, assign to him as many objects as possible (starting from the one with lowest load
    # going to the heaviest one, i.e. sort s first! BEWARE, NEED TO KEEP ALSO INITIAL INDEX IN ORDER TO USE PROPER INDEX IN D MATRIX) When he can't take any more object: continue with next courier etc. -> end up with a possible assignment
    # Then compute distances (can do also while assign objects to couriers! All in a single pass) and find the maximum.
    # Complexity: O(n) NICE!
    # Question: ask if you can do this! Pro: would make binary search start way lower! Also could add the initial constraint AllLessEq(distances, upper_bound) because right now doing so it's useless
    # TODO: Do as line above, add constriant AllLessEq(distances, upper_bound)
    # IMP: note that such a solution would NOT be possible if the ACTUAL LOADS ARE NOT SORTED! TODO: find an initial solution that satisfies this!
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])

    ## VARIABLE
    # distances_i = representation in binary of the distance travelled by courier i
    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    ## CONSTRAINT
    # definition of distances using constraints
    for i in range(m):
        solver.add(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    # TODO: remove
    print(f"Finished constraint 'distances_def' at time {round(time.time() - start_time, 1)}s")


    model = None
    obj_value = None
    encoding_time = time.time()
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")

    timeout = encoding_time + timeout_duration

    # TODO: remove
    # if now >= timeout:
    #     return "Encoding time too slow"

    solver.push()

    if search == 'Linear':

        solver.set('timeout', millisecs_left(time.time(), timeout))
        while solver.check() == z3.sat:

            model = solver.model()
            obj_value = obj_function(model, distances)
            print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")


            if obj_value <= lower_bound:
                break

            upper_bound = obj_value - 1
            upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
                
            solver.pop()
            solver.push()

            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))

    # TODO: try to skew the search towards lower values, like by not dividing by 2 every time but like doing 1/4 from lower_bound, or sqrt of difference
    # (...but it's really theoretically wrong because switching point (minimum) is really uniformly distributed!)
    elif search == 'Binary':

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T_compressed = evaluate(model, t_compressed)
        A = evaluate(model, a)
        Dists = evaluate(model, distances)
        displayMCP(T_compressed, Dists, obj_value, A)

    return (obj_value, solving_time)


In [ ]:
def multiple_couriers_planning_3(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)
                    
    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    # r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    # t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)]
    # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    solver = Solver()


    ## CONSTRAINTS
    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)
        # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
        solver.add(sort_decreasing(courier_loads))
        # Break symmetry within same load amounts, i.e.:
        # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
        # i.e. the second courier will be the one assigned to the route containing the item with lower index j
        for i in range(m - 1):
            solver.add(
                Implies(equal(courier_loads[i], courier_loads[i + 1]),
                        leq(a[i], a[i + 1])))

    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

    # TODO: remove
    print(f"Finished constraint 1 at time {round(time.time() - start_time, 1)}s")


    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
        solver.add(leq(courier_loads[i], l_bin[i]))

    # TODO: remove
    print(f"Finished constraint 2 at time {round(time.time() - start_time, 1)}s")


    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    for i in range(m):
        solver.add(at_least_one_seq(a[i]))

    # TODO: remove
    print(f"Finished constraint 3 at time {round(time.time() - start_time, 1)}s")


    # Constraint 4: routes

    # for i in range(m):
    #     # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
    #     solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

    #     # Constraint 4.2: row j has a 1 iff courier i delivers object j
    #     # rows
    #     for j in range(n):
    #         solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
    #         solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
    #     solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

    #     # Constraint 4.3: column j has a 1 iff courier i delivers object j
    #     # columns
    #     for k in range(n):
    #         solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
    #         solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k)
    #     solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin

    #     # Constraint 4.4: HAMILTONIAN CYCLES
    #     # Uses matrix t[i] in order to check that the adjacency matrix r[i] represents a Hamiltonian cycle
    #     for j in range(n):
    #         # assignments implications
    #         solver.add(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) => [t_ijk = True, for some k] (i delivers j at some time k)
    #         solver.add(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) => [t_ijk = False, forall k] (i never delivers j)

    #         # can't deliver two items at the same time
    #         solver.add(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

    #         # 1st trip
    #         solver.add(Implies(r[i][n][j], t[i][j][0])) # r_inj (i rides n (origin) --> j) => t_ij0 (i delivers j as 1st item)

    #         # successive trips
    #         for h in range(1, n):
    #             # The next two constraints make a double implication together: (Exists k s.t. i came from that k to j AND he delivered k at time h-1) <==> (j delivered at time h)
    #             # ==>
    #             for k in range(n):
    #                 solver.add(Implies(And(r[i][k][j], t[i][k][h-1]), t[i][j][h]))  # r_i,k,j AND t_i,k,h-1 (i goes from k to j AND he delivered k at time h-1) => t_i,j,h (j delivered at time h)

    #             # <==
    #             solver.add(Implies(t[i][j][h], Or([And(r[i][k][j], t[i][k][h-1]) for k in range(n)])))       # if t_i,j,h (j delivered at time h) => [r_i,k,j AND t_i,k,h-1. for some k] (i came from some k to j AND he delivered that k at time h-1)

    # TODO: remove
    print(f"Finished constraint 4 + Hamiltonian at time {round(time.time() - start_time, 1)}s")


    ## OPTIMIZATION SEARCH

    # flatten r and D
    # flat_r = [flatten(r[i]) for i in range(m)]
    # flat_D = flatten(D)
    # # convert flat_D to binary
    # flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]


    # Constraint 5: distances travelled by each courier
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    # max_distances = [max(D[i]) for i in range(n+1)]
    # max_distances.sort()
    # upper_bound = sum(max_distances[m-1:])
    # # TODO: maybe we can find a tighter upper bound on obj func, i.e. start from a naive solution, that is:
    # # starting from the courier with lowest load, assign to him as many objects as possible (starting from the one with lowest load
    # # going to the heaviest one, i.e. sort s first! BEWARE, NEED TO KEEP ALSO INITIAL INDEX IN ORDER TO USE PROPER INDEX IN D MATRIX) When he can't take any more object: continue with next courier etc. -> end up with a possible assignment
    # # Then compute distances (can do also while assign objects to couriers! All in a single pass) and find the maximum.
    # # Complexity: O(n) NICE!
    # # Question: ask if you can do this! Pro: would make binary search start way lower! Also could add the initial constraint AllLessEq(distances, upper_bound) because right now doing so it's useless
    # # TODO: Do as line above, add constriant AllLessEq(distances, upper_bound)
    # # IMP: note that such a solution would NOT be possible if the ACTUAL LOADS ARE NOT SORTED! TODO: find an initial solution that satisfies this!
    # lower_bound = max([D[n][j] + D[j][n] for j in range(n)])

    # distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    # # definition of distances using constraints
    # for i in range(m):
    #     solver.add(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    # TODO: remove
    print(f"Finished distances def constraint at time {round(time.time() - start_time, 1)}s")


    model = None
    obj_value = None
    encoding_time = time.time()
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")

    timeout = encoding_time + timeout_duration

    # # TODO: remove
    # # if now >= timeout:
    # #     return "Encoding time too slow"

    # solver.push()

    # if search == 'Linear':

    #     solver.set('timeout', millisecs_left(time.time(), timeout))
    #     while solver.check() == z3.sat:

    #         model = solver.model()
    #         obj_value = obj_function(model, distances)
    #         print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")


    #         if obj_value <= lower_bound:
    #             break

    #         upper_bound = obj_value - 1
    #         upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


    #         solver.pop()
    #         solver.push()

    #         solver.add(AllLessEq_bin(distances, upper_bound_bin))
    #         now = time.time()
    #         if now >= timeout:
    #             break
    #         solver.set('timeout', millisecs_left(now, timeout))


    # elif search == 'Binary':

    #     upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
    #     solver.add(AllLessEq_bin(distances, upper_bound_bin))

    #     lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
    #     solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    #     while lower_bound <= upper_bound:
    #         mid = int((lower_bound + upper_bound)/2)
    #         mid_bin = int_to_bin(mid, num_bits(mid))
    #         solver.add(AllLessEq_bin(distances, mid_bin))

    #         now = time.time()
    #         if now >= timeout:
    #             break
    #         solver.set('timeout', millisecs_left(now, timeout))
    #         print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

    #         if solver.check() == z3.sat:
    #             model = solver.model()
    #             obj_value = obj_function(model, distances)
    #             print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

    #             if obj_value <= 1:
    #                 break

    #             upper_bound = obj_value - 1
    #             upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


    #         else:
    #             print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

    #             lower_bound = mid + 1
    #             lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

    #         solver.pop()
    #         solver.push()
    #         solver.add(AllLessEq_bin(distances, upper_bound_bin))
    #         solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    # else:
    #     raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")

    print(solver.check())
    model = solver.model()
    print(evaluate(model, a))
    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    # if model is None:
    #     return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    # R = evaluate(model, r)
    # assert(check_all_hamiltonian(R))

    # if display_solution:
    #     T = evaluate(model, t)
    #     Dists = evaluate(model, distances)
    #     displayMCP(T, Dists, obj_value)

    return (obj_value, solving_time)


In [ ]:
def multiple_couriers_planning_4(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)
                    
    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)]
    # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    solver = Solver()


    ## CONSTRAINTS
    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)
        # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
        solver.add(sort_decreasing(courier_loads))
        # Break symmetry within same load amounts, i.e.:
        # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
        # i.e. the second courier will be the one assigned to the route containing the item with lower index j
        for i in range(m - 1):
            solver.add(
                Implies(equal(courier_loads[i], courier_loads[i + 1]),
                        leq(a[i], a[i + 1])))

    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

    # TODO: remove
    print(f"Finished constraint 1 at time {round(time.time() - start_time, 1)}s")


    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
        solver.add(leq(courier_loads[i], l_bin[i]))

    # TODO: remove
    print(f"Finished constraint 2 at time {round(time.time() - start_time, 1)}s")


    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    for i in range(m):
        solver.add(at_least_one_seq(a[i]))

    # TODO: remove
    print(f"Finished constraint 3 at time {round(time.time() - start_time, 1)}s")


    # Constraint 4: routes

    for i in range(m):
        ## Constraints on r
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        ### TODO: see if implicit or redundant constraints
        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k)
        solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin
        ### end TODO

        # Constraints on t
        for j in range(n-1):
            # when starts false -> all false afterwards
            solver.add(Implies(all_false([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])))

        for j in range(n):
            # assignments implications
            solver.add(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) => [t_ijk = True, for some k] (i delivers j at some time k)
            solver.add(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) => [t_ijk = False, forall k] (i never delivers j)

            # can't deliver two items at the same time
            solver.add(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))


        # channeling constraints
        for j in range(n):

            # 1st trip
            solver.add(r[i][n][j] == t[i][j][0]) # r_inj (i rides n (origin) <--> j) => t_ij0 (i delivers j as 1st item)

            for h in range(1, n):
                for k in range(n):
                    solver.add(Implies(And(t[i][j][h-1], t[i][k][h]), r[i][j][k]))
                # Could add other (implicit?) constraint here

                # Find the last object delivered and place 1 in returning edge
                solver.add(Implies(And(at_least_one_seq([t[i][k][j] for k in range(n)]), all_false([t[i][k][j] for k in range(n)])), equal([t[i][k][j] for k in range(n)], [r[i][k][n] for k in range(n)])))






    # TODO: remove
    print(f"Finished constraint 4 + Hamiltonian at time {round(time.time() - start_time, 1)}s")


    ## OPTIMIZATION SEARCH

    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]


    # Constraint 5: distances travelled by each courier
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])
    # TODO: maybe we can find a tighter upper bound on obj func, i.e. start from a naive solution, that is:
    # starting from the courier with lowest load, assign to him as many objects as possible (starting from the one with lowest load
    # going to the heaviest one, i.e. sort s first! BEWARE, NEED TO KEEP ALSO INITIAL INDEX IN ORDER TO USE PROPER INDEX IN D MATRIX) When he can't take any more object: continue with next courier etc. -> end up with a possible assignment
    # Then compute distances (can do also while assign objects to couriers! All in a single pass) and find the maximum.
    # Complexity: O(n) NICE!
    # Question: ask if you can do this! Pro: would make binary search start way lower! Also could add the initial constraint AllLessEq(distances, upper_bound) because right now doing so it's useless
    # TODO: Do as line above, add constriant AllLessEq(distances, upper_bound)
    # IMP: note that such a solution would NOT be possible if the ACTUAL LOADS ARE NOT SORTED! TODO: find an initial solution that satisfies this!
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])

    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    # definition of distances using constraints
    for i in range(m):
        solver.add(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    # TODO: remove
    print(f"Finished distances def constraint at time {round(time.time() - start_time, 1)}s")


    model = None
    obj_value = None
    encoding_time = time.time()
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")

    timeout = encoding_time + timeout_duration

    # TODO: remove
    # if now >= timeout:
    #     return "Encoding time too slow"

    solver.push()

    if search == 'Linear':

        solver.set('timeout', millisecs_left(time.time(), timeout))
        while solver.check() == z3.sat:

            model = solver.model()
            obj_value = obj_function(model, distances)
            print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")


            if obj_value <= lower_bound:
                break

            upper_bound = obj_value - 1
            upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            solver.pop()
            solver.push()

            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))


    elif search == 'Binary':

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T = evaluate(model, t)
        Dists = evaluate(model, distances)
        displayMCP(T, Dists, obj_value)

    return (obj_value, solving_time)


### Testing zone

In [ ]:
import os
import sys
import numpy as np

# Brainstorm:
# to benchmark k models over n instances: for every instance i, run all k models and record their performance
# -> better to use something like bash? or just run them from another python program? I imagine every model as a .py file importing 
# utils and encodings from respective files depending on ints needs, while all instances just being in a folder full of .dat files

def run_model_on_instance(MCP_model, file, **kwargs):
    """Read the instance from .dat file and run the given MCP model on it

    Args:
        MCP_model (function): function executing the SAT-encoding and solving of the given instance
        file (str): path of the .dat file representing the instance
    """
    with open(file) as f:
        m = int(next(f))
        n = int(next(f))
        l = [int(e) for e in next(f).split()]
        s = [int(e) for e in next(f).split()]
        D = np.genfromtxt(f, dtype=int).tolist()
        
    return MCP_model(m, n, l, s, D, **kwargs)

# TODO: remove num_instances as a param, it's too specific (maybe instances folder changes etc.)
def test_model(MCP_model, output_file, num_instances, **kwargs):
    old_stdout = sys.stdout
    with open(output_file, 'w') as f:
        # TODO: remove stdout redirection, just needed to print partial solutions as well
        sys.stdout = f
        # TODO: make more general with path etc.
        for file in os.listdir('../instances/')[:num_instances]:
            print(f"======================{file}======================", file=f)
            print(f"{file} -> {run_model_on_instance(MCP_model, f'../instances/{file}', **kwargs)}\n", file=f)
            print(f"Finito {file}", file=old_stdout)
        sys.stdout = old_stdout

def compare_models(MCP_modelA, MCP_modelB, modelA_name, modelB_name, num_instances, **kwargs):
    old_stdout = sys.stdout
    with open(f'{modelA_name}_vs_{modelB_name}', 'w') as f:
        # TODO: remove stdout redirection, just needed to print partial solutions as well
        sys.stdout = f
        # TODO: make more general with path etc.
        for file in os.listdir('../instances/')[:num_instances]:
            print(f"======================{file}======================", file=f)

            print(f"----------------------{modelA_name}---------------------", file=f)
            print(f"{file} -> {run_model_on_instance(MCP_modelA, f'../instances/{file}', **kwargs)}\n", file=f)

            print(f"----------------------{modelB_name}---------------------", file=f)
            print(f"{file} -> {run_model_on_instance(MCP_modelB, f'../instances/{file}', **kwargs)}\n", file=f)

            print(f"Finito {file}", file=old_stdout)
        sys.stdout = old_stdout


In [ ]:
run_model_on_instance(multiple_couriers_planning_1, '../instances/inst07.dat', search='Binary', display_solution=False)

In [ ]:
run_model_on_instance(multiple_couriers_planning_4, '../instances/inst02.dat', search='Binary', display_solution=True)

In [ ]:
run_model_on_instance(multiple_couriers_planning_1, '../instances/inst02.dat', search='Binary', display_solution=True)

In [ ]:
run_model_on_instance(multiple_couriers_planning_1, '../instances/inst02.dat', symmetry_breaking=False, search='Binary', display_solution=True)

In [ ]:
run_model_on_instance(multiple_couriers_planning_2, '../instances/inst02.dat', symmetry_breaking=False, search='Binary', display_solution=True)

In [ ]:
run_model_on_instance(multiple_couriers_planning_4, '../instances/inst13.dat', symmetry_breaking=False, search='Linear', display_solution=True)

In [ ]:
run_model_on_instance(multiple_couriers_planning_3, '../instances/inst13.dat', search='Binary', display_solution=False)

In [ ]:
run_model_on_instance(multiple_couriers_planning_3, '../instances/inst13.dat', symmetry_breaking=False, search='Binary', display_solution=False)

In [ ]:
run_model_on_instance(multiple_couriers_planning_1, '../instances/inst13.dat', symmetry_breaking=True, search='Binary', display_solution=False)

In [ ]:
run_model_on_instance(multiple_couriers_planning_1, '../instances/inst13.dat', symmetry_breaking=False, search='Binary', display_solution=False)

In [ ]:
run_model_on_instance(multiple_couriers_planning_2, '../instances/inst13.dat', symmetry_breaking=True, search='Binary', display_solution=False)

In [ ]:
run_model_on_instance(multiple_couriers_planning_2, '../instances/inst13.dat', symmetry_breaking=False, search='Binary', display_solution=False)

In [ ]:
test_model(multiple_couriers_planning_1, 'model1.txt', 15, display_solution=True)

In [ ]:
compare_models(multiple_couriers_planning_1, multiple_couriers_planning_2, 'model1', 'model2_NoEncodingTime', 8, display_solution=True) 

# Unit testing area (to be removed)

In [ ]:
# # Testing of leq
# for i in range(1, 16):
#     for j in range(7, 9):
#         digits_i = num_bits(i)
#         i_bin = int_to_bin(i, digits_i)
#         digits_j = num_bits(j)
#         j_bin = int_to_bin(j, digits_j)

#         s = Solver()

#         s.add(leq(i_bin, j_bin))
#         if s.check() == z3.sat:
#             m = s.model()
#             print(f"{i} <= {j}")
#         else:
#             print(f"{i} > {j}")


In [ ]:
# # Testing of sum_bin with variable num_digits
# for i in range(1, 10):
#     for j in range(100, 110):
#         digits_i = num_bits(i)
#         i_bin = int_to_bin(i, digits_i)
#         digits_j = num_bits(j) + 1
#         j_bin = int_to_bin(j, digits_j)
#         d_bin = [Bool(f"d_{k}") for k in range(digits_j)]

#         s = Solver()

#         s.add(sum_bin(i_bin, j_bin, d_bin, f"Prova_{i}_{j}"))
#         if s.check() == z3.sat:
#             m = s.model()
#             d = bin_to_int([1 if m.evaluate(d_bin[k]) else 0 for k in range(digits_j)])
#             print(f"{i} + {j} = {d}")
#         else:
#             print(f"UNSAT with {i} + {j}")


In [ ]:
# # testing of conditional_sum_K_bin
# n = 4
# x = [Bool(f'x_{i}') for i in range(n)]
# alpha = [7, 9, 11, 4]
# digits = num_bits(max(e for e in alpha))
# alpha_bin = [int_to_bin(e, digits) for e in alpha]

# for comb in range(2**n):
#     s = Solver()
#     somma = 0
#     for i in range(n):
#         # print(comb, i, comb % (2**i))
#         if comb >= (2**i) and comb % (2**(i+1)) == 1:
#             print(comb, 2**i)
#             s.add(x[i])
#             somma += alpha[i]
#         else:
#             s.add(Not(x[i]))

#     delta = [Bool(f'delta_{j}') for j in range(8)]
#     s.add(conditional_sum_K_bin(x, alpha_bin, delta, 'ciao'))
#     s.check()
#     m = s.model()
#     print(f"{somma} ?= {bin_to_int([1 if m.evaluate(e) else 0 for e in delta])}")


# # print(s.check())
# # m = s.model()
# # print([m.evaluate(e) for e in x])
# # print([m.evaluate(e) for e in delta])

In [ ]:
# # Testing of LinearInteger_bin
# n = 4
# digits = 6
# x = [Bool(f'x_{i}') for i in range(n)]
# alpha = [7, 9, 11, 6]
# alpha_bin = [int_to_bin(e, digits) for e in alpha]
# beta = int_to_bin(5, digits)
# s = Solver()
# s.add(LinearInteger_bin(x, alpha_bin, beta, 'ciao'))
# s.add(Or(x))

# print(s.check())
# m = s.model()
# print([m.evaluate(e) for e in x])
